In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [3]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [4]:
import torch
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import random
import pandas as pd

# Assuming x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled, x_test_scaled, y_test_scaled,
# input_size, output_size, test_window_size, scaler are available


# Hyperparameter search space
search_space = {
    'learning_rate': [0.00005],
    'window_size': [ 5],
    'hidden_dim': [256],
    'n_layers': [11],
    'batch_evaluation_frequency': [3, 4, 5]
}

#Best Parameters: {'learning_rate': 5e-05, 'window_size': 5, 'hidden_dim': 256,
#                  'n_layers': 11, 'batch_evaluation_frequency': 4, 'test_loss': 6463.20556640625}


# Number of random search iterations
num_iterations = 10

best_params = None

for _ in range(num_iterations):
    params = {
        'learning_rate': random.choice(search_space['learning_rate']),
        'window_size': random.choice(search_space['window_size']),
        'hidden_dim': random.choice(search_space['hidden_dim']),
        'n_layers': random.choice(search_space['n_layers']),
        'batch_evaluation_frequency': random.choice(search_space['batch_evaluation_frequency'])
    }

    model = LSTMModel(input_size, params['hidden_dim'], params['n_layers'], output_size)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Training using Walk-Forward Validation
    for i in range(params['window_size'], len(x_train)):
        optimizer.zero_grad()

        start_idx = i - params['window_size']
        end_idx = i
        x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
        y_window = torch.tensor(y_train[start_idx:end_idx], dtype=torch.float32)
        x_window = x_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        outputs, hidden = model(x_window, hidden)
        loss = criterion(outputs, y_window)

        loss.backward()
        optimizer.step()

        if i % params['batch_evaluation_frequency'] == 0:
            with torch.no_grad():
                x_val_window = torch.tensor(x_val[:params['window_size']], dtype=torch.float32)
                y_val_window = torch.tensor(y_val[:params['window_size']], dtype=torch.float32)
                x_val_window = x_val_window.view(1, params['window_size'], input_size)

                hidden = model.init_hidden(1)
                val_outputs, _ = model(x_val_window, hidden)
                val_loss = criterion(val_outputs, y_val_window)

                if i % 1000 == 0:  # Print every 1000 iterations
                    print(f"Iteration {i}, Train Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    # Test set evaluation
    with torch.no_grad():
        x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)
        y_test_window = torch.tensor(y_test[:params['window_size']], dtype=torch.float32)
        x_test_window = x_test_window.view(1, params['window_size'], input_size)

        hidden = model.init_hidden(1)
        test_outputs, _ = model(x_test_window, hidden)
        test_loss = criterion(test_outputs, y_test_window)

    print(f"Test Loss: {test_loss.item()}")

    # Update the best_params if the current model performs better on the test set
    if best_params is None:
        best_params = params
        best_params['test_loss'] = test_loss.item()
    elif test_loss < best_params['test_loss']:
        best_params = params
        best_params['test_loss'] = test_loss.item()

print(f"Best Parameters: {best_params}")

C:\Users\pitr7\AppData\Local\Temp\ipykernel_10592\600017660.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_window = torch.tensor(x_train[start_idx:end_idx], dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\pitr7\AppData\Local\Temp\ipykernel_10592\600017660.py:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_val_window = torch.tensor(x_val[

Iteration 1000, Train Loss: 1.0489604473114014, Validation Loss: 2.2498157024383545
Iteration 2000, Train Loss: 2.075005292892456, Validation Loss: 2.2356631755828857
Iteration 3000, Train Loss: 0.23920011520385742, Validation Loss: 2.2373318672180176
Iteration 4000, Train Loss: 0.378330796957016, Validation Loss: 2.2188894748687744
Iteration 5000, Train Loss: 0.3043554723262787, Validation Loss: 2.221320152282715
Iteration 6000, Train Loss: 0.16968679428100586, Validation Loss: 2.2248661518096924
Iteration 7000, Train Loss: 0.23000600934028625, Validation Loss: 2.2298691272735596
Iteration 8000, Train Loss: 0.3340556025505066, Validation Loss: 2.222750186920166


C:\Users\pitr7\AppData\Local\Temp\ipykernel_10592\600017660.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test_window = torch.tensor(x_test[:params['window_size']], dtype=torch.float32)


Test Loss: 0.41795605421066284
Iteration 3000, Train Loss: 0.23816022276878357, Validation Loss: 2.237905979156494
Iteration 6000, Train Loss: 0.16921117901802063, Validation Loss: 2.225233554840088
Test Loss: 0.4182061553001404
Iteration 1000, Train Loss: 1.0484105348587036, Validation Loss: 2.249269962310791
Iteration 2000, Train Loss: 2.0730135440826416, Validation Loss: 2.234947681427002
Iteration 3000, Train Loss: 0.24075278639793396, Validation Loss: 2.2364869117736816
Iteration 4000, Train Loss: 0.3784767985343933, Validation Loss: 2.2188034057617188
Iteration 5000, Train Loss: 0.3053213953971863, Validation Loss: 2.2208151817321777
Iteration 6000, Train Loss: 0.1702841818332672, Validation Loss: 2.2244038581848145
Iteration 7000, Train Loss: 0.23146991431713104, Validation Loss: 2.229116201400757
Iteration 8000, Train Loss: 0.3345987796783447, Validation Loss: 2.2225394248962402
Test Loss: 0.4171517789363861
Iteration 1000, Train Loss: 1.0492819547653198, Validation Loss: 2.250

In [18]:
# Assuming 'data' is your original DataFrame containing the time series data
print(data.tail(5))


             date    open    high     low   close      volume  adjusted_close  \
10742  2023-07-25  193.33  194.44  192.92  193.62  37283200.0        193.3589   
10743  2023-07-26  193.67  195.64  193.32  194.50  47471900.0        194.2377   
10744  2023-07-27  196.02  197.20  192.55  193.22  47460200.0        192.9594   
10745  2023-07-28  194.67  196.63  194.14  195.83  48291400.0        195.5659   
10746  2023-07-31  196.06  196.49  195.26  196.45  38824100.0        196.1851   

       change_percent  avg_vol_20d  
10742            0.45   52369165.0  
10743            0.45   52206220.0  
10744           -0.66   52018390.0  
10745            1.35   52115595.0  
10746            0.32   49803320.0  


In [17]:
# Assuming 'window_size' is a parameter in your code
params['window_size'] = 5  # Change the window_size to 5

# Assuming x_train, y_train, x_val, y_val, x_test, y_test, input_size, output_size, LSTMModel are defined

# Set the model's mode to evaluation before the rolling predictions
model.eval()

# Number of future days to predict
num_future_days = 5  # You can adjust this based on your requirement

# Initialize the window with the last values from the training set
rolling_window = x_train[-params['window_size']:]
rolling_window = rolling_window.view(1, params['window_size'], input_size)

# Print the initial rolling window values and dimensions
print("Initial Rolling Window Values (Original):", scaler_x.inverse_transform(rolling_window.view(-1, input_size)))
print("Initial Rolling Window Dimensions:", rolling_window.shape)

# Perform rolling predictions
for _ in range(num_future_days):
    with torch.no_grad():
        # Predict the next day's close value
        hidden = model.init_hidden(1)
        prediction, _ = model(rolling_window, hidden)

        # Convert the prediction to a numpy array
        prediction_np = prediction.numpy()

        # Print the predicted close value for the first day after existing data
        predicted_close = scaler_y.inverse_transform(prediction_np)[0, 0]
        print("Predicted Close Value for the First Day after Existing Data (Original):", predicted_close)

        # Print dimensions before concatenation
        print("Dimensions Before Concatenation - Rolling Window:", rolling_window.shape)
        print("Dimensions Before Concatenation - Prediction:", prediction_np.shape)

        # Update the rolling window for the next iteration
        rolling_window = torch.cat((rolling_window[:, 1:, :], torch.tensor(prediction_np).view(1, 1, -1)), dim=1)

        # Print the entire rolling window
        print("Rolling Window After Prediction (Original):", scaler_x.inverse_transform(rolling_window.view(-1, input_size)))

        # Print the first rolling window after starting with predictions
        if _ == 0:
            print("First Rolling Window After Starting with Predictions (Original):", scaler_x.inverse_transform(rolling_window.view(-1, input_size)))


Initial Rolling Window Values (Original): [[ 2.21983979e+01  2.28479971e+01  2.20528002e+01  4.69919920e+06
   3.42700308e-01  1.87000004e+00  2.07898037e+08]
 [ 2.30496039e+01  2.39120000e+01  2.29376033e+01  6.22159998e+06
   3.56800340e-01  8.50000004e-01  3.56143759e+08]
 [ 2.54911971e+01  2.54912027e+01  2.49311975e+01  6.70369970e+06
   3.78700292e-01 -1.69000008e+00  2.84303878e+08]
 [ 3.12479993e+01  3.31295967e+01  3.12480012e+01  1.13840055e+06
   1.13800448e-01  5.65999996e+00  2.25719198e+08]
 [ 1.87739997e+02  1.90010800e+02  1.87300398e+02  2.16101999e+07
   5.76009959e+00  2.14999995e+00  5.13932433e+08]]
Initial Rolling Window Dimensions: torch.Size([1, 5, 7])
Predicted Close Value for the First Day after Existing Data (Original): 116.24349
Dimensions Before Concatenation - Rolling Window: torch.Size([1, 5, 7])
Dimensions Before Concatenation - Prediction: (1, 1)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 7 but got size 1 for tensor number 1 in the list.

In [16]:
# Assuming 'window_size' is a parameter in your code
params['window_size'] = 5  # Change the window_size to 5

# Assuming x_train, y_train, x_val, y_val, x_test, y_test, input_size, output_size, LSTMModel are defined

# Set the model's mode to evaluation before the rolling predictions
model.eval()

# Number of future days to predict
num_future_days = 5  # You can adjust this based on your requirement

# Initialize the window with the last values from the training set
rolling_window = x_train[-params['window_size']:]
rolling_window = rolling_window.view(1, params['window_size'], input_size)

# Print the initial rolling window values and dimensions
print("Initial Rolling Window Values:", rolling_window)
print("Initial Rolling Window Dimensions:", rolling_window.shape)

# Perform rolling predictions
for _ in range(num_future_days):
    with torch.no_grad():
        # Predict the next day's close value
        hidden = model.init_hidden(1)
        prediction, _ = model(rolling_window, hidden)

        # Convert the prediction to a numpy array
        prediction_np = prediction.numpy()

        # Print the predicted close value for the first day after existing data
        print("Predicted Close Value for the First Day after Existing Data:", scaler_y.inverse_transform(prediction_np)[0, 0])

        # Print dimensions before concatenation
        print("Dimensions Before Concatenation - Rolling Window:", rolling_window.shape)
        print("Dimensions Before Concatenation - Prediction:", prediction_np.shape)

        # Update the rolling window for the next iteration
        rolling_window = torch.cat((rolling_window[:, 1:, :], torch.tensor(prediction_np).view(1, 1, -1)), dim=1)

        # Print the entire rolling window
        print("Rolling Window After Prediction:", rolling_window)

        # Print the first rolling window after starting with predictions
        if _ == 0:
            print("First Rolling Window After Starting with Predictions:", rolling_window)


Initial Rolling Window Values: tensor([[[-0.6824, -0.6819, -0.6794, -0.5100, -0.4514,  0.6253, -0.4263],
         [-0.6759, -0.6739, -0.6726, -0.4536, -0.4510,  0.2627,  0.1182],
         [-0.6574, -0.6620, -0.6573, -0.4358, -0.4505, -0.6401, -0.1456],
         [-0.6137, -0.6046, -0.6089, -0.6418, -0.4574,  1.9724, -0.3608],
         [ 0.5734,  0.5753,  0.5865,  0.1160, -0.3108,  0.7248,  0.6978]]])
Initial Rolling Window Dimensions: torch.Size([1, 5, 7])
Predicted Close Value for the First Day after Existing Data: 116.24349
Dimensions Before Concatenation - Rolling Window: torch.Size([1, 5, 7])
Dimensions Before Concatenation - Prediction: (1, 1)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 7 but got size 1 for tensor number 1 in the list.

In [14]:
# Assuming 'window_size' is a parameter in your code
params['window_size'] = 5  # Change the window_size to 5

# Assuming x_train, y_train, x_val, y_val, x_test, y_test, input_size, output_size, LSTMModel are defined

# Set the model's mode to evaluation before the rolling predictions
model.eval()

# Number of future days to predict
num_future_days = 5  # You can adjust this based on your requirement

# Initialize the window with the last values from the training set
rolling_window = x_train[-params['window_size']:]
rolling_window = rolling_window.view(1, params['window_size'], input_size)

# Perform rolling predictions
for _ in range(num_future_days):
    with torch.no_grad():
        # Predict the next day's close value
        hidden = model.init_hidden(1)
        prediction, _ = model(rolling_window, hidden)

        # Convert the prediction to a numpy array
        prediction_np = prediction.numpy()

        # Print the predicted close value for the first day after existing data
        print("Predicted Close Value for the First Day after Existing Data:", scaler_y.inverse_transform(prediction_np)[0, 0])

        # Print dimensions before concatenation
        print("Dimensions Before Concatenation - Rolling Window:", rolling_window.shape)
        print("Dimensions Before Concatenation - Prediction:", prediction_np.shape)

        # Update the rolling window for the next iteration
        rolling_window = torch.cat((rolling_window[:, 1:, :], torch.tensor(prediction_np).view(1, 1, -1)), dim=1)

        # Print the entire rolling window
        print("Rolling Window After Prediction:", rolling_window)

        # Print the first rolling window after starting with predictions
        if _ == 0:
            print("First Rolling Window After Starting with Predictions:", rolling_window)


Predicted Close Value for the First Day after Existing Data: 116.24349
Dimensions Before Concatenation - Rolling Window: torch.Size([1, 5, 7])
Dimensions Before Concatenation - Prediction: (1, 1)


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 7 but got size 1 for tensor number 1 in the list.

In [5]:
# Assuming x_train, y_train, x_val, y_val, x_test, y_test, input_size, output_size, LSTMModel are defined

# Set the model's mode to evaluation before the rolling predictions
model.eval()

# Number of future days to predict
num_future_days = 5  # You can adjust this based on your requirement

# Initialize the window with the last values from the training set
rolling_window = x_train[-params['window_size']:]
rolling_window = rolling_window.view(1, params['window_size'], input_size)

# Perform rolling predictions
for _ in range(num_future_days):
    with torch.no_grad():
        # Predict the next day's close value
        hidden = model.init_hidden(1)
        prediction, _ = model(rolling_window, hidden)

        # Convert the prediction to a numpy array
        prediction_np = prediction.numpy()

        # Add the predicted value to the rolling window
        rolling_window = torch.cat((rolling_window[:, 1:, :], torch.tensor(prediction_np).view(1, 1, -1)), dim=1)

        # Print or store the predicted value
        print("Predicted Close Value:", scaler_y.inverse_transform(prediction_np)[0, 0])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 7 but got size 1 for tensor number 1 in the list.